In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn


C:\Users\Jacob\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels = 1)
    image_resized = tf.image.resize_images(image_decoded, [920, 920])
    return image_resized, label

In [6]:
# import data from generated images and csv files
numTrials = 1
numTimeSteps = 20
dt = .5
for trial in range(numTrials):
    directory = r"C:/dev/starTrack/data/trial" + str(trial) + "/"
    
    # input data filenames
    inputFilenameList = []
    tInit = 0.0
    inputFP = r"satFOV/"
    for j in range(numTimeSteps):
        t = tInit + .5*j
        filename = directory + inputFP + "t" + str(t) + ".jpg"
        inputFilenameList.append(filename) 
    
    inputFilenames = tf.constant(inputFilenameList)
    
    # output data 
    outputFilename = "rotational_dynamics.csv"
    outputFilepath = directory + outputFilename
    outputData = pd.read_csv(outputFilepath)
    
    # label the images with angular velocities
    omega = np.zeros((3, numTimeSteps))
    #TODO: the next dataset won't have the space in front of the header names, so remove it
    omegaX = outputData[' omegaX'].as_matrix()
    omegaY = outputData[' omegaY'].as_matrix()
    omegaZ = outputData[' omegaZ'].as_matrix()
    
    # labels[:,i] is the label for the image in filenames[i]
    for j in range(numTimeSteps):
            omega[0,j] = omegaX[j]
            omega[1,j] = omegaY[j]
            omega[2,j] = omegaZ[j]
    omega = np.reshape(omega, (3, numTimeSteps))
    labels = tf.constant(omega.T)
    dataset = tf.data.Dataset.from_tensor_slices((inputFilenames, labels))
    dataset = dataset.map(_parse_function)

In [8]:
print(dataset.output_shapes)

print(dataset.output_types)

(TensorShape([Dimension(920), Dimension(920), Dimension(1)]), TensorShape([Dimension(3)]))
(tf.float32, tf.float64)


In [9]:
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)
'''

#define constants
#unrolled through 28 time steps
time_steps = numTimeSteps

#hidden LSTM units
num_units=128

#rows of 28 pixels
n_input=28

#learning rate for adam
learning_rate=0.001

#mnist is meant to be classified in 10 classes(0-9).
n_classes=10

#size of batch
batch_size=128


In [10]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)


In [ ]:

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
